In [1]:
import requests
import json
import urllib.parse
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from tabulate import tabulate
import datetime as dt
import os 
import yaml

# pathing 
CUR_DIR = os.getcwd()
DATA_DIR = os.path.join(CUR_DIR,'data')
DATA_DIR

# creds 
with open('creds/api-key.yaml', 'r') as file: 
    data = yaml.full_load(file)['elastic-email']['shared-account']

APP_PASSWORD = data['app-password']
API_KEY = data['ee-api-key']
SENDER_EMAIL = data['sender-email']
RECEIVER_EMAIL = data['receiver-email']

# general email settings 
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_SUBJECT = f'Your survey result for {dt.datetime.now().strftime("%B %d, %Y")}'

In [69]:
data


{'sender-email': 'dil.techimpact@gmail.com',
 'receiver-email': 'apsocarras@gmail.com',
 'login-password': 'career1Stop2023@AK',
 'app-password': 'lmaa umxm bufl hyba',
 'ee-api-key': '734F795D3B03858E78DA0575E91A93BB626C97BC09FEB1F51DA20C026992F23E8FB125763B1C03704C6CC6D4BF7D8944'}

In [63]:
class ApiClient:
    apiUri = 'https://api.elasticemail.com/v2'

    def __init__(self, api_key):
        self.apiKey = api_key

    def Request(self, method, url, data):
        data['apikey'] = self.apiKey
        if method == 'POST':
            result = requests.post(f'{self.apiUri}{url}', data=data)
        elif method == 'PUT':
            result = requests.put(f'{self.apiUri}{url}', data=data)
        elif method == 'GET':
            result = requests.get(f'{self.apiUri}{url}', params=data)

        jsonMy = result.json()

        if jsonMy['success'] is False:
            return jsonMy['error']

        return jsonMy['data']

def Send(subject, EEfrom, fromName, to, bodyHtml, bodyText, isTransactional, api_key):
    client = ApiClient(api_key)
    return client.Request('POST', '/email/send', {
        'subject': subject,
        'from': EEfrom,
        'fromName': fromName,
        'to': to,
        'bodyHtml': bodyHtml,
        'bodyText': bodyText,
        'isTransactional': isTransactional
    })

In [34]:
# Create hyperlinks to job page
def create_hyperlink(rec:dict) -> str:
    base_url = "https://www.careeronestop.org/Toolkit/Careers/Occupations/occupation-profile.aspx?"
    parameters = {
        "keyword": urllib.parse.quote(rec['OccupationTitle']),
        "location": "US",
        "lang": "en",
        "onetCode": rec["OnetCode"]
    }
    url_params = "&".join([f"{key}={value}" for key, value in parameters.items()])
    url = f'{base_url}{url_params}'
    return f'<a href="{url}">{rec["OccupationTitle"]}</a>'

In [51]:
# Load data
with open(os.path.join(DATA_DIR, 'skills-submit-api', 'example-response.json'), 'r') as file: 
    cos_response = json.load(file)

# Extract and format data
rec_list = cos_response['SKARankList']
rename_keys_map = {'Rank': 'Your Match Rank', 
                   'OccupationTitle': 'Occupation Title', 
                   'AnnualWages': 'Average Wages (Annual)', 
                   'TypicalEducation': 'Typical Education'
                   }
for rec in rec_list:
    # Set occupation title to hyperlink
    rec['OccupationTitle'] = create_hyperlink(rec)
    # Drop redundant field (used to create the hyperlink)
    rec.pop('OnetCode')
    # Format wages 
    rec['AnnualWages'] = f"${rec['AnnualWages']:,.0f}"
    # Rename columns
    for k,v in rename_keys_map.items(): 
        rec[v] = rec.pop(k) # rename the key 
       
## Format HTML table 
column_alignments = {
    "Your Match Rank": "center",
    "Occupation Title": "left",
    "Average Wages (Annual)": "right",
    "Typical Education": "left",
    "Outlook": "left"
}

table = tabulate(
    rec_list,
    headers="keys",
    tablefmt="html",
    colalign=[column_alignments[col] for col in column_alignments.keys()]
)

In [67]:
## Styling the table 
table_headers = column_alignments.keys()
table_html = "<table>"
table_html += "<tr>"

# Header Style
header_style = "font-weight: bold; font-size: 20px;"
for header in table_headers:
    table_html += f"<th style='{header_style}'>{header}</th>"
table_html += "</tr>"

# Cell Style
cell_style = "font-weight: normal; font-size: 16px;"  
for rec in rec_list:
    table_html += "<tr>"
    for header in table_headers:
        table_html += f"<td style='{cell_style}'>{rec[header]}</td>"
    table_html += "</tr>"
table_html += "</table>"

# Message Style
message_style = "font-weight: normal; font-size: 16px;"

# Use the table_html in your email body
message = f'<div style="{message_style}">Your customized top 20 suggestion list is as follows:</div>\n\n{table_html}'

pd.read_html(message)[0] # What the table will look like in your email


/var/folders/qy/qp1k23kj0jj0bpkn10sx9mhh0000gn/T/ipykernel_9636/1639357725.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(message)[0] # What message will look like in your email


,Your Match Rank,Occupation Title,Average Wages (Annual),Typical Education,Outlook
0,1,Physical Medicine and Rehabilitation Physicians,"$223,410",Doctoral or professional degree,Below Average
1,2,Advanced Practice Psychiatric Nurses,"$81,220",Bachelor's degree,Bright
2,3,Naturopathic Physicians,"$106,230",Master's degree,Below Average
3,4,Acupuncturists,"$72,220",Master's degree,Average
4,5,"Anthropology and Archeology Teachers, Postseco...","$85,000",Doctoral or professional degree,Bright
...,...,...,...,...,...
145,146,"Elementary School Teachers, Except Special Edu...","$61,690",Bachelor's degree,Bright
146,147,Agricultural Technicians,"$41,760",Associate's degree,Bright
147,148,"Library Science Teachers, Postsecondary","$76,370",Doctoral or professional degree,Bright
148,149,"Directors, Religious Activities and Education","$49,380",Bachelor's degree,Average


In [68]:
# Send the Email
result = Send(EMAIL_SUBJECT,
              SENDER_EMAIL,
              "Tech Impact",
              RECEIVER_EMAIL,
              f"<h1>{message}</h1>",
              f"{message}",
              True,
              API_KEY)

print(result)

{'transactionid': 'c517a305-9a6b-8117-e005-5ea630b1cdb8', 'messageid': 'JR-0AtBBkR__UH9RR30pyw2'}
